In [7]:
import os
import json
import pandas as pd
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer

In [2]:
# Load the answer key to work with
with open(os.path.join(os.getcwd(), "ce311k_answers", "00_variables_raw.json")) as f:
    answer_key = json.load(f)

del answer_key["cells"][0:2]

In [3]:
# Get the data all cleaned up
questions_ordered = {}
q_num = -1
question, solution, test = False, False, False
for item in answer_key["cells"]:
    # Set the question number
    if item["cell_type"] == "raw":
        if item["source"][0] == "# BEGIN QUESTION\n":
            q_num += 1
    # Ensure naming sequence is created
    if f"q{q_num}" not in questions_ordered.keys():
        questions_ordered[f"q{q_num}"] = {}
        questions_ordered[f"q{q_num}"]["question"] = None
        questions_ordered[f"q{q_num}"]["solution"] = None
        questions_ordered[f"q{q_num}"]["test"] = None
    # Denote what type of info the line is
    if item["cell_type"] == "raw":
        if item["source"][0] == "# BEGIN QUESTION\n":
            question = True
        elif item["source"][0] == "# BEGIN SOLUTION":
            solution = True
        elif item["source"][0] == "# BEGIN TESTS":
            test = True
    # Put the info into the correct place
    elif item["cell_type"] != "raw":
        if question:
            questions_ordered[f"q{q_num}"]["question"] = item["source"]
            question = False
        elif solution:
            questions_ordered[f"q{q_num}"]["solution"] = item["source"][1:-1]
            solution = False
        elif test:
            questions_ordered[f"q{q_num}"]["test"] = item["source"]
            test = False

In [ ]:
# Get data set up into dataframe for vectorization
df = pd.DataFrame(
    {
        "question": [
            questions_ordered[i]["question"] for i in questions_ordered.keys()
        ],
        "solution": [
            questions_ordered[i]["solution"] for i in questions_ordered.keys()
        ],
        "test": [questions_ordered[i]["test"] for i in questions_ordered.keys()],
    }
)

# Define the embedding function
model = SentenceTransformer(
    "all-MiniLM-L6-v2"
)  # This is the default embedding function for chroma
questions = []
solutions = []
tests = []
for i in questions_ordered.keys():
    questions.append(questions_ordered[i]["question"])
    solutions.append(questions_ordered[i]["solution"])
    tests.append(questions_ordered[i]["test"])

question_embeddings = []
solution_embeddings = []
test_embeddings = []
for i in range(len(questions)):
    question_embeddings.append(model.encode(questions[i]))
for i in range(len(solutions)):
    solution_embeddings.append(model.encode(solutions[i]))
for i in range(len(tests)):
    test_embeddings.append(model.encode(tests[i]))

df["question_embeddings"] = question_embeddings
df["solution_embeddings"] = solution_embeddings
df["test_embeddings"] = test_embeddings

In [ ]:
# Set up the chroma client
chroma_client = chromadb.PersistentClient(path="./chroma_save_states")
collection = chroma_client.get_or_create_collection(name="00_variables")

# Add the data to the chroma client
metas = []
ids = []
for i in range(len(df.index)):
    metas.append({"00_variables": i})
for i in range(len(df.index)*3):
    ids.append(f'id{i}')

collection.add(
    documents=df["question"].tolist() + df["solution"].tolist() + df["test"].tolist(),
    embeddings=df["question_embeddings"].tolist() + df["solution_embeddings"].tolist() + df["test_embeddings"].tolist(),
    metadatas=metas + metas + metas,
    ids=ids,
)